In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import os

# Создаём директорию для графиков, если её ещё нет
plots_dir = "/home/ubuntu/plots"
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Загружаем датасет
data_path = "processed_data_new.csv"
df = pd.read_csv(data_path)

# Определяем целевые столбцы
target_columns = [
    "Вопрос решен",
    "Нравится качество выполнения заявки",
    "Нравится качество работы сотрудников",
    "Нравится скорость отработки заявок",
    "Понравилось выполнение заявки",
    "Ничего из перечисленного"
]

# Определяем столбец с текстом
text_column = "comment"

# Обрабатываем возможные NaN в текстовом столбце
df[text_column] = df[text_column].fillna("")

# --- 1. Построение распределения классов ---
print("Генерация графика распределения классов...")
plt.figure(figsize=(12, 8))
df[target_columns].sum().sort_values(ascending=False).plot(kind='bar')
plt.title('Распределение комментариев по категориям (во всём датасете)')
plt.ylabel('Количество комментариев')
plt.xlabel('Категории')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
class_distribution_path = os.path.join(plots_dir, "class_distribution.png")
plt.savefig(class_distribution_path)
plt.close()
print(f"График распределения классов сохранён в {class_distribution_path}")

# Подготавливаем признаки (X) и цели (y)
X = df[text_column]
y = df[target_columns]

# Разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаём пайплайн для векторизации текста и классификации
model_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=5000)),
    ("clf", OneVsRestClassifier(LogisticRegression(solver="liblinear", random_state=42)))
])

# Обучаем модель
print("\nОбучение модели...")
model_pipeline.fit(X_train, y_train)
print("Обучение модели завершено.")

# Делаем предсказания (вероятности для ROC-AUC)
print("\nВыполнение предсказаний...")
y_pred_proba = model_pipeline.predict_proba(X_test)
print("Предсказания выполнены.")

# --- 2. Построение ROC-кривых ---
print("\nГенерация графика ROC-кривых...")
plt.figure(figsize=(12, 10))

roc_auc_scores = {}
for i, label in enumerate(target_columns):
    if len(np.unique(y_test[label])) > 1:  # Проверяем, есть ли положительные примеры
        fpr, tpr, _ = roc_curve(y_test[label], y_pred_proba[:, i])
        roc_auc = roc_auc_score(y_test[label], y_pred_proba[:, i])
        roc_auc_scores[label] = roc_auc
        plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.2f})')
        print(f"ROC-AUC для {label}: {roc_auc:.4f}")
    else:
        roc_auc_scores[label] = np.nan
        print(f"ROC-AUC для {label}: Невычислима (в y_test только один класс)")

plt.plot([0, 1], [0, 1], 'k--')  # Пунктирная диагональ
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Доля ложноположительных срабатываний (FPR)')
plt.ylabel('Доля истинно положительных срабатываний (TPR)')
plt.title('ROC-кривые для каждой категории')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
roc_curves_path = os.path.join(plots_dir, "roc_curves.png")
plt.savefig(roc_curves_path)
plt.close()
print(f"График ROC-кривых сохранён в {roc_curves_path}")

# Вычисляем средний ROC-AUC (макро-среднее, игнорируя NaN)
average_roc_auc = np.nanmean(list(roc_auc_scores.values()))
print(f"\nСредний ROC-AUC (макро): {average_roc_auc:.4f}")

print("\nСкрипт с построением графиков завершён.")
